# Data Cleaning:

In [21]:
import pandas as pd
import numpy as np
import re
import unidecode
import nltk
import glob
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from wordcloud import WordCloud, STOPWORDS
from textblob import TextBlob

path = '/home/thomas/Desktop/Thomas/Python/oscar_winning/actor_sheets/'

    all data is imported from individual .csv files from a local directory and concatenated into a single DataFrame.

In [2]:
df = pd.concat((pd.read_csv(f) for f in glob.glob(path+"*.csv"))).reset_index(drop=True).rename(columns={'AUDIENCE_SCORE':'SCORE_1','CRITIC_SCORE':'SCORE_2'})

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14474 entries, 0 to 14473
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ACTOR        14474 non-null  object
 1   TITLE        14474 non-null  object
 2   ROLE         14474 non-null  object
 3   SCORE_1      14474 non-null  int64 
 4   SCORE_2      14474 non-null  int64 
 5   REVIEWS      7872 non-null   object
 6   NUM_REVIEWS  14474 non-null  int64 
dtypes: int64(3), object(4)
memory usage: 791.7+ KB


In [4]:
df.head(5)

,ACTOR,TITLE,ROLE,SCORE_1,SCORE_2,REVIEWS,NUM_REVIEWS
0,Nigel Hawthorne,Call Me Claus,\n Nick (Character)\n ...,0,0,'With Brian Stokes Mitchell and Victor Garber ...,1
1,Nigel Hawthorne,Victoria & Albert,\n Lord William Lamb (C...,0,0,NaN,0
2,Nigel Hawthorne,The Clandestine Marriage,\n Lord Ogleby (Charact...,0,0,NaN,0
3,Nigel Hawthorne,A Reasonable Man,\n Judge Wendon (Charac...,0,0,NaN,0
4,Nigel Hawthorne,The Winslow Boy,\n Arthur Winslow (Char...,77,97,"""What began as a transformation of borrowed ma...",8


    The first step is to remove all newline characters and begin splitting and formatting the string information: The reviews column has most of the punctuation stripped and is summarily split into a list. The role column is split at either a right paranethesis or a comma and converts the string to lowercase unicode characters. The column for the actor name converts the string to unicode characters only.

In [5]:
df = df.replace('\n','', regex=True)
df.REVIEWS = df.REVIEWS.fillna('nothing')
df.REVIEWS = df.REVIEWS.apply(lambda x: re.sub('”|“','"',str(x)).replace("''",'""').replace('\'','"').replace('", "','\\').replace('"s',"'s").split('\\'))
df['ROLE'] = df['ROLE'].str.split('\(|,')
df.TITLE = df.TITLE.apply(lambda x: re.sub('\.\.\.|\:|\;|\,|\.|\'', '', unidecode.unidecode(str(x).strip())).lower())
df.ACTOR = df.ACTOR.apply(lambda x: unidecode.unidecode(str(x)))

In [6]:
df

,ACTOR,TITLE,ROLE,SCORE_1,SCORE_2,REVIEWS,NUM_REVIEWS
0,Nigel Hawthorne,call me claus,"[ Nick , Character) ...",0,0,"[""With Brian Stokes Mitchell and Victor Garber...",1
1,Nigel Hawthorne,victoria & albert,"[ Lord William Lamb , C...",0,0,[nothing],0
2,Nigel Hawthorne,the clandestine marriage,"[ Lord Ogleby , Charact...",0,0,[nothing],0
3,Nigel Hawthorne,a reasonable man,"[ Judge Wendon , Charac...",0,0,[nothing],0
4,Nigel Hawthorne,the winslow boy,"[ Arthur Winslow , Char...",77,97,"[""What began as a transformation of borrowed m...",8
...,...,...,...,...,...,...,...
14469,Terrence Howard,best laid plans,"[ Jimmy , Character) ...",38,43,"["", , , Best Laid Plans is so concerned with b...",5
14470,Terrence Howard,butter,"[ Dexter Banks , Charac...",0,0,[nothing],0
14471,Terrence Howard,johns,"[ Jimmy the Warlock , C...",58,54,"["", David Arquette and Lukas Haas find the rig...",2
14472,Terrence Howard,sunset park,"[ Spaceman , Character)...",83,13,"["", The camera fails to capture the basketball...",3


    The role column is further split into to categories where there is only one item or more. In each case further removal of punctuation is done and the first element is stripped of any invisible characters and taken as the role* for that film.
    
    *In all cases, due to how the data is initially presented, a character role is always listed before other credits.
    
    This information is then recombined into a single DataFrame.

In [7]:
char_roles = df[df.ROLE.str.len() >1].copy()
non_char = df[df.ROLE.str.len() == 1].copy()

char_roles.ROLE = char_roles.ROLE.apply(lambda x: x[0].lower())
char_roles.ROLE = char_roles.ROLE.apply(lambda x: re.sub('\,|\.|\"|\'','',unidecode.unidecode(str(x).strip())).lower())
non_char.ROLE = non_char.ROLE.apply(lambda x: re.sub('\,|\.|\"|\'','',unidecode.unidecode(x[0].strip())).lower())

cleaned_df = pd.concat([char_roles, non_char],axis=0)
cleaned_df

,ACTOR,TITLE,ROLE,SCORE_1,SCORE_2,REVIEWS,NUM_REVIEWS
0,Nigel Hawthorne,call me claus,nick,0,0,"[""With Brian Stokes Mitchell and Victor Garber...",1
1,Nigel Hawthorne,victoria & albert,lord william lamb,0,0,[nothing],0
2,Nigel Hawthorne,the clandestine marriage,lord ogleby,0,0,[nothing],0
3,Nigel Hawthorne,a reasonable man,judge wendon,0,0,[nothing],0
4,Nigel Hawthorne,the winslow boy,arthur winslow,77,97,"[""What began as a transformation of borrowed m...",8
...,...,...,...,...,...,...,...
14292,Max von Sydow,rosamunde pilchner two sisters,director,0,0,[nothing],0
14298,Max von Sydow,europa,narrator,89,80,"[""The German moral tragedy of the Nazi era is ...",2
14304,Max von Sydow,katinka,director,0,0,[nothing],0
14364,Ralph Fiennes,andre the voice of wine,narrator,0,0,[nothing],0


    The same process as above is repeated for an additional data set containing the actor name, role, title of movie and award nomination/win information. 

In [8]:
df_award = pd.read_csv("~/Desktop/Thomas/Python/oscar_winning/best_actor_data.csv")
df_award = df_award.replace('\n','', regex=True)
df_award

,ACTOR,ROLE,TITLE,AWARD
0,Adam Driver,Charlie Barber,Marriage Story,1
1,Adolphe Menjou,Walter Burns,The Front Page,1
2,Adrien Brody,Władysław Szpilman,The Pianist,2
3,Al Pacino,Frank Serpico,Serpico,1
4,Al Pacino,Michael Corleone,The Godfather Part II,1
...,...,...,...,...
451,William Powell,Godfrey Park,My Man Godfrey,1
452,William Powell,Clarence Day Sr.,Life with Father,1
453,Woody Allen,Alvy Singer,Annie Hall,1
454,Woody Harrelson,Larry Flynt,The People vs. Larry Flynt,1


In [9]:
df_award.ACTOR = df_award.ACTOR.apply(lambda x: unidecode.unidecode(str(x)))
df_award.ROLE = df_award.ROLE.apply(lambda x: re.sub('\,|\.|\"|\'','',unidecode.unidecode(str(x))).strip().lower())
df_award.TITLE = df_award.TITLE.apply(lambda x: re.sub('\.\.\.|\:|\;|\,|\.|\'','',unidecode.unidecode(str(x).strip()).lower()))
df_award

,ACTOR,ROLE,TITLE,AWARD
0,Adam Driver,charlie barber,marriage story,1
1,Adolphe Menjou,walter burns,the front page,1
2,Adrien Brody,wladyslaw szpilman,the pianist,2
3,Al Pacino,frank serpico,serpico,1
4,Al Pacino,michael corleone,the godfather part ii,1
...,...,...,...,...
451,William Powell,godfrey park,my man godfrey,1
452,William Powell,clarence day sr,life with father,1
453,Woody Allen,alvy singer,annie hall,1
454,Woody Harrelson,larry flynt,the people vs larry flynt,1


    Both data sets are combined using an outer join*, so that the award column can have all Nan values filled with 0 completing the multiclass labeling.
    
    Due to availablity of information 3 titles and the respective information/lackthereof are lost during this join.

In [10]:
full_df = pd.merge(df_award, cleaned_df, how='outer', on=['ACTOR','TITLE','ROLE'])
full_df.AWARD = full_df.AWARD.fillna(0)
full_df = full_df.dropna()
full_df.head()

,ACTOR,ROLE,TITLE,AWARD,SCORE_1,SCORE_2,REVIEWS,NUM_REVIEWS
0,Adam Driver,charlie barber,marriage story,1.0,85.0,94.0,"[""Noah Baumbach's latest film portrays both si...",48.0
1,Adolphe Menjou,walter burns,the front page,1.0,60.0,93.0,"[""What truth it contains has been thrown into ...",1.0
2,Adrien Brody,wladyslaw szpilman,the pianist,2.0,96.0,95.0,"[""Old-fashioned in both visual and narrative s...",40.0
3,Al Pacino,frank serpico,serpico,1.0,88.0,90.0,"[""Wonderful potential, and wasted. Serpico has...",6.0
4,Al Pacino,michael corleone,the godfather part ii,1.0,97.0,98.0,"[""In sheer physical terms, however, Godfather ...",17.0


    A separate DataFrame is created where each list of reviews is split into columns*.
    
    *The highest number of reviews is 66 and yet there are 79 columns, this implies that I was not able to correctly identify all possible cases for splitting appropriately. This is likely due to inconsistent usage of quotation and apostrophe marks as well as aberrant punctuation.

In [13]:
text_analysis =full_df['REVIEWS'].apply(pd.Series).add_prefix('REVIEW_').fillna('neutral')
text_analysis.head()

,REVIEW_0,REVIEW_1,REVIEW_2,REVIEW_3,REVIEW_4,REVIEW_5,REVIEW_6,REVIEW_7,REVIEW_8,REVIEW_9,...,REVIEW_69,REVIEW_70,REVIEW_71,REVIEW_72,REVIEW_73,REVIEW_74,REVIEW_75,REVIEW_76,REVIEW_77,REVIEW_78
0,"""Noah Baumbach's latest film portrays both sid...","At its best, this movie is searing. It pinpoin...","The entire cast is rock solid, with Johansson ...","It's a triumph, a mix of pleasure and pain tha...","Johansson and Driver are remarkably, heartbrea...","By the end, you""re rooting for both Charlie an...","It's not quite as bleak as a Bergman film, alt...",You may find yourself crying for them occasion...,"I""m not buying every chapter of this Marriage ...","Noah Baumbach is the writer-director, and to w...",...,neutral,neutral,neutral,neutral,neutral,neutral,neutral,neutral,neutral,neutral
1,"""What truth it contains has been thrown into t...",neutral,neutral,neutral,neutral,neutral,neutral,neutral,neutral,neutral,...,neutral,neutral,neutral,neutral,neutral,neutral,neutral,neutral,neutral,neutral
2,"""Old-fashioned in both visual and narrative st...","The loneliness, guilt and impotence on Brody's...",It is unwatchably harrowing. And the images of...,Brody is a sublimely haunting presence at the ...,It's Roman Polanski's strongest and most perso...,In going home to tell Szpilman's story Polansk...,We admire this film for its harsh objectivity ...,Brody tracks Szpilman's descent from smug cele...,A portrait of hell so shattering it's impossib...,The Pianist is a devastating story of survival...,...,neutral,neutral,neutral,neutral,neutral,neutral,neutral,neutral,neutral,neutral
3,"""Wonderful potential, and wasted. Serpico has ...",Sidney Lumet's direction adeptly combines grit...,A virtuoso performance by Al Pacino and some e...,"Another problem, these days, is Pacino's chara...","Lumet's biopic of Frank Serpico, the virtuous ...",A remarkable record of one man's rebellion aga...,neutral,neutral,neutral,neutral,...,neutral,neutral,neutral,neutral,neutral,neutral,neutral,neutral,neutral,neutral
4,"""In sheer physical terms, however, Godfather P...",Few movie sequels are as good as the films the...,It goes much deeper than The Godfather in anal...,"An admirable, responsible production, less emo...",One of the greatest sequels ever.,"It is even better than the first film, and has...",Either taken alone or together with the origin...,One of the most ambitious and brilliantly exec...,This is quite simply one of the saddest movies...,Not once does Pacino overtly ask for the audie...,...,neutral,neutral,neutral,neutral,neutral,neutral,neutral,neutral,neutral,neutral


    Sentiment Analysis using NLTK and TextBlob is done on each column entry. The resulting tuples are split into two separate DataFrames where a best attempt at providing correct mean values is done by replacing all 0's with np.nan. The columns containing these mean values are then added back to the main DataFrame.

In [14]:
col_list = ['REVIEW_{}'.format(n) for n in range(79)]
stoppers = stopwords.words('english')

for col in col_list:
    text_analysis[col] = text_analysis[col].apply(lambda x: " ".join(x.lower() for x in x.split()))
    text_analysis[col] = text_analysis[col].apply(lambda x: " ".join(x for x in x.split() if x not in stoppers))

In [15]:
def senti_analysis(x):
    return list(TextBlob(x).sentiment)

for col in col_list:
    col_name = '{}_senti'.format(col)
    full_df[col_name] = text_analysis[col].apply(senti_analysis)
    
    
senti_list = ['{}_senti'.format(col) for col in col_list]
pol_name_dict = {'{}'.format(item): '{}_polarity'.format(item) for item in senti_list}
obj_name_dict = {'{}'.format(item): '{}_objectivity'.format(item) for item in senti_list}
polarity_df = full_df[senti_list].apply(lambda x:x.str[0])
polarity_df.rename(columns=pol_name_dict, inplace=True)
objectivity_df = full_df[senti_list].apply(lambda x: x.str[1])
objectivity_df.rename(columns=obj_name_dict, inplace=True)
polarity_df = polarity_df.replace(0, np.nan)
objectivity_df = objectivity_df.replace(0,np.nan)
polarity_df['polarity_mean'] = polarity_df.mean(axis = 1, skipna=True)
objectivity_df['objectivity_mean'] = objectivity_df.mean(axis = 1, skipna=True)

In [16]:
full_df = full_df[['ACTOR','ROLE','TITLE','AWARD','SCORE_1','SCORE_2','NUM_REVIEWS']]
pol_out = polarity_df['polarity_mean'].copy()
obj_out = objectivity_df['objectivity_mean'].copy()
full_df = pd.concat([full_df,pd.concat([pol_out,obj_out],axis=1)],axis=1)
full_df


,ACTOR,ROLE,TITLE,AWARD,SCORE_1,SCORE_2,NUM_REVIEWS,polarity_mean,objectivity_mean
0,Adam Driver,charlie barber,marriage story,1.0,85.0,94.0,48.0,0.190969,0.479507
1,Adolphe Menjou,walter burns,the front page,1.0,60.0,93.0,1.0,0.500000,1.000000
2,Adrien Brody,wladyslaw szpilman,the pianist,2.0,96.0,95.0,40.0,0.128576,0.603966
3,Al Pacino,frank serpico,serpico,1.0,88.0,90.0,6.0,0.238542,0.572917
4,Al Pacino,michael corleone,the godfather part ii,1.0,97.0,98.0,17.0,0.380286,0.646926
...,...,...,...,...,...,...,...,...,...
14472,Max von Sydow,director,rosamunde pilchner two sisters,0.0,0.0,0.0,0.0,NaN,NaN
14473,Max von Sydow,narrator,europa,0.0,89.0,80.0,2.0,0.500000,0.562500
14474,Max von Sydow,director,katinka,0.0,0.0,0.0,0.0,NaN,NaN
14475,Ralph Fiennes,narrator,andre the voice of wine,0.0,0.0,0.0,0.0,NaN,NaN


In [17]:
full_df = full_df.fillna(0)
full_df

,ACTOR,ROLE,TITLE,AWARD,SCORE_1,SCORE_2,NUM_REVIEWS,polarity_mean,objectivity_mean
0,Adam Driver,charlie barber,marriage story,1.0,85.0,94.0,48.0,0.190969,0.479507
1,Adolphe Menjou,walter burns,the front page,1.0,60.0,93.0,1.0,0.500000,1.000000
2,Adrien Brody,wladyslaw szpilman,the pianist,2.0,96.0,95.0,40.0,0.128576,0.603966
3,Al Pacino,frank serpico,serpico,1.0,88.0,90.0,6.0,0.238542,0.572917
4,Al Pacino,michael corleone,the godfather part ii,1.0,97.0,98.0,17.0,0.380286,0.646926
...,...,...,...,...,...,...,...,...,...
14472,Max von Sydow,director,rosamunde pilchner two sisters,0.0,0.0,0.0,0.0,0.000000,0.000000
14473,Max von Sydow,narrator,europa,0.0,89.0,80.0,2.0,0.500000,0.562500
14474,Max von Sydow,director,katinka,0.0,0.0,0.0,0.0,0.000000,0.000000
14475,Ralph Fiennes,narrator,andre the voice of wine,0.0,0.0,0.0,0.0,0.000000,0.000000


    All NaN values are replaced with Zero, and then a final preprocessing of the actor names is added as an additional piece of information. This final DataFrame is then saved and used in the analysis.

In [18]:
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()

In [19]:
full_df['NUMERIC_KEY'] = label_encoder.fit_transform(full_df['ACTOR'])
full_df

,ACTOR,ROLE,TITLE,AWARD,SCORE_1,SCORE_2,NUM_REVIEWS,polarity_mean,objectivity_mean,NUMERIC_KEY
0,Adam Driver,charlie barber,marriage story,1.0,85.0,94.0,48.0,0.190969,0.479507,0
1,Adolphe Menjou,walter burns,the front page,1.0,60.0,93.0,1.0,0.500000,1.000000,1
2,Adrien Brody,wladyslaw szpilman,the pianist,2.0,96.0,95.0,40.0,0.128576,0.603966,2
3,Al Pacino,frank serpico,serpico,1.0,88.0,90.0,6.0,0.238542,0.572917,3
4,Al Pacino,michael corleone,the godfather part ii,1.0,97.0,98.0,17.0,0.380286,0.646926,3
...,...,...,...,...,...,...,...,...,...,...
14472,Max von Sydow,director,rosamunde pilchner two sisters,0.0,0.0,0.0,0.0,0.000000,0.000000,141
14473,Max von Sydow,narrator,europa,0.0,89.0,80.0,2.0,0.500000,0.562500,141
14474,Max von Sydow,director,katinka,0.0,0.0,0.0,0.0,0.000000,0.000000,141
14475,Ralph Fiennes,narrator,andre the voice of wine,0.0,0.0,0.0,0.0,0.000000,0.000000,169


In [20]:
full_df.to_csv('processed_data.csv', index=False)